<a href="https://colab.research.google.com/github/sb09ii/credit-scoring/blob/main/credit-scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
if HAS_XGB:
param_grid['XGBoost'] = {
'clf__n_estimators': [100, 200],
'clf__max_depth': [3, 5]
}


best_score = -1
best_model = None


for name, clf in models.items():
print(f"\nTraining {name}...")
pipe = Pipeline(steps=[('preprocessor', preprocessor), ('clf', clf)])


if name in param_grid:
grid = GridSearchCV(pipe, param_grid[name], cv=5, scoring='roc_auc', n_jobs=-1)
grid.fit(X_train, y_train)
model = grid.best_estimator_
print(f"Best params for {name}: {grid.best_params_}")
else:
pipe.fit(X_train, y_train)
model = pipe


score = evaluate_model(model, X_test, y_test)
if score > best_score:
best_score = score
best_model = model


print(f"\nBest Model Selected with score {best_score:.4f}")
return best_model


def save_model(model, filename=MODEL_FILE):
joblib.dump(model, filename)
print(f"Model saved to {filename}")


def load_model(filename=MODEL_FILE):
return joblib.load(filename)


def predict_example(model, sample):
df_sample = pd.DataFrame([sample])
proba = None
try:
proba = model.predict_proba(df_sample)[:,1][0]
except Exception:
pass
pred = model.predict(df_sample)[0]
print(f"Predicted class: {pred}, Probability of default: {proba if proba else 'N/A'}")
return pred


def main():
df = load_data()
eda(df)


preprocessor, X, y = preprocess(df, target_col='target')
best_model = train_and_select_best(preprocessor, X, y)


save_model(best_model)


print("\n-- Example Prediction --")
sample = {col: X.iloc[0][col] for col in X.columns}
model = load_model()
predict_example(model, sample)


if __name__ == '__main__':
main()